# Getting 3,200 tweets of all users

The script below demonstrates how to collect 3,200 tweets per user. 

- I use a few screen names ('realdonaldtrump', 'hillaryclinton, 'kstatebusiness') for the demonstration purpose. 
- If you understand this script and need to collect 3,200 tweets for a large number of Twitter users, you should use **3_get_all_users_tweets_final_s.py**

In [ ]:
# Adapted from https://gist.github.com/yanofsky/5436496#file-tweet_dumper-py-L36
import tweepy #https://github.com/tweepy/tweepy
import csv

#Variables that contains the user credentials to access Twitter API

consumer_key = 'insert your own key here'
consumer_secret = 'insert your own key here'
access_token = 'insert your own key here'
access_token_secret = 'insert your own key here'

#Twitter only allows access to a users most recent 3240 tweets with this method

#authorize twitter, initialize tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)

#initialize a list to hold all the tweepy Tweets

screen_names = ['realdonaldtrump','KStateBusiness','HillaryClinton']

writefile=open("data/output_user_tweets.csv", "wb")
w=csv.writer(writefile)

for i in screen_names:
    alltweets = []	
    print i
    
    new_tweets = api.user_timeline(screen_name = i, count=200)

    #save most recent tweets
    alltweets.extend(new_tweets)

    try:
        oldest = alltweets[-1].id - 1
    except:
        pass
    
    #keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:
        print "getting tweets before %s" % (oldest)

        #all subsiquent requests use the max_id param to prevent duplicates
        new_tweets = api.user_timeline(screen_name = i,count=200,max_id=oldest)

        #save most recent tweets
        alltweets.extend(new_tweets)

        #update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1

        print "...%s tweets downloaded so far" % (len(alltweets))

    #write the csv	
    for tweet in alltweets:
        w.writerow([i, tweet.id_str, tweet.created_at, tweet.text.encode("utf-8")])
        #w.writerow([i, tweet.id_str, tweet.created_at, tweet.text.encode("utf-8"), tweet.source])
        
writefile.close()

# Get all followers of a Twitter user

Let's suppose that you want to find out who is following you, a company, or a popular sport team.

1. Then, first you should collect all followers of a user using this script (we'll collect their Twitter screen names). Save the entire screen names in csv.

2. Second, you use the python script (3_get_all_users_tweets.py) to all tweets per each follower.

In [ ]:
import time
import tweepy
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

#Variables that contains the user credentials to access Twitter API

consumer_key = 'insert your own key here'
consumer_secret = 'insert your own key here'
access_token = 'insert your own key here'
access_token_secret = 'insert your own key here'

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth, wait_on_rate_limit=True)

# get the followers' screen names (K-State Provost)
provost_followers = []
for user in tweepy.Cursor(api.followers, screen_name="KStateProvost", count=200, 
                          wait_on_rate_limit=True, wait_on_rate_limit_notify=True).items():
    provost_followers.append(user.screen_name)
    print user.screen_name
len(provost_followers)

In [ ]:
import csv

writefile=open("data/screen_names.csv", "wb")
w=csv.writer(writefile)
for i in provost_followers:
    w.writerow([i])
writefile.close()

# Appendix: Get all followers of a Twitter user (To Avoid Twitter's Rate Limits through Sleeping)

- If you want to collect a complete list of Twitter follwers for some popular businesses or people, then use the following script (**4_Collecting_LargeFollowers_s.py**)

In [ ]:
import time
import csv
import tweepy
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

#Variables that contains the user credentials to access Twitter API
consumer_key = ''
consumer_secret = ''
access_token = ''
access_token_secret = ''

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

#save the screen names
writefile = open("data/screen_names_large.csv", "wb")
w = csv.writer(writefile)

users = tweepy.Cursor(api.followers, screen_name='GTMSportswear').items()

while True:
    try:
        user = next(users)
    except tweepy.TweepError:
        time.sleep(60*15)
        try:
            user = next(users)
        #http://stackoverflow.com/questions/30582332/python-tweeperror-failed-to-send-request
        except BaseException as e:
            print('failed on_status,',str(e)) # print the error code obtained from twitter
            time.sleep(5) # provide a time before resuming the code when an error arises
    except StopIteration:
        break
    print user.screen_name
    w.writerow([user.screen_name])

writefile.close()